In [4]:
import json
import os

In [6]:
with open('mysoalan.com') as fopen:
    data = json.load(fopen)
    
len(data)

2437

In [7]:
mapping = {}
with open('mysoalan.com-fitz.jsonl') as fopen:
    for l in fopen:
        l = json.loads(l)
        mapping[l['file']] = l['text']

In [8]:
filtered = []
for d in data:
    if d['pdfLink'] is None:
        continue
    file = os.path.join('mysoalan.com-pdf', os.path.split(d['pdfLink'])[1])
    if not os.path.exists(file):
        continue
    if file not in mapping:
        continue
    d['local_filename'] = file
    d['texts'] = mapping[file]
    filtered.append(d)

In [9]:
len(filtered)

1680

In [10]:
# import fitz

# doc = fitz.open(filtered[0]['local_filename'])
# page = doc.load_page(0)
# pixmap = page.get_pixmap(dpi=100)
# img = pixmap.tobytes()
# pixmap.save('test.jpg')

In [11]:
filtered[0]['texts'][0]

'                                                                                                                                                                                                                                               SULIT \n2 \n \n \n \n \n \n \n \n \n \n \n                  SULIT \nBahagian  A \n[30  markah] \nJawab  semua  soalan \n \n                 Soalan 1 berdasarkan peta topo siri 2280/2/2019.Kaji peta ini dengan teliti sebelum menjawab soalan    \n    \n \n                   berikut. \n                                                                    58 \n                                                                  \n \n \n \n \n                                                                    56 \n                                                                          04                        06 \n \n1.  (a) Berapakah ketinggian Bkt.Tukang dalam segi empat grid di atas? \n \n..................................................................

In [12]:
!rm -rf generate-qa
!mkdir generate-qa

In [13]:
results = []
for i in range(len(filtered)):
    for t in filtered[i]['texts']:
        s = f"""
        ```
        {t}
        ```

        convert into JSON, [{{'question', 'choices'}}]
        """.strip()
        results.append(s)
        
len(results)

27066

In [29]:
from openai import OpenAI

def answer(q, i):
    filename = f'generate-qa/{i}.json'
    if os.path.exists(filename):
        return
    
    openai = OpenAI(
        base_url='http://localhost:8005/v1',
        api_key='empty',
    )
    completion = openai.chat.completions.create(
        model='huseinzol05/Meta-Llama-3-8B-Instruct-fix-tokenizer',
        messages=[{'role': 'user', 'content': q}],
        temperature=0.9,
        max_tokens=1024,
        top_p=0.95,
    )
    r = completion.choices[0].message.content
    
    with open(filename, 'w') as fopen:
        json.dump(r, fopen)

In [15]:
openai = OpenAI(
    base_url='http://localhost:8005/v1',
    api_key='empty',
)

In [16]:
completion = openai.chat.completions.create(
    model='huseinzol05/Meta-Llama-3-8B-Instruct-fix-tokenizer',
    messages=[{'role': 'user', 'content': 'hello'}],
    temperature=0.9,
    max_tokens=1024,
    top_p=0.95,
)

In [26]:
def consumer(queue, name):
    while True:
        if queue.qsize() == 0:
            break
        item = queue.get()
        answer(*item)
    print(f'consumer {name} done')

In [27]:
urls = [(q, no) for no, q in enumerate(results)]

In [30]:
answer(*urls[10003])

In [32]:
from threading import Thread
from queue import Queue
from tqdm import tqdm

queue = Queue()
for u in urls:
    queue.put(u)
    
ori_size = queue.qsize()

In [33]:
max_worker = 50
consumers = [Thread(target=consumer, args=(queue,i)) for i in range(max_worker)]
for i in range(len(consumers)):
    consumers[i].start()
    
pbar = tqdm(total=ori_size)
last_size = 0
while True:
    size = queue.qsize()
    if size == 0:
        break
    left = ori_size - size
    minus = left - last_size
    if minus > 0:
        pbar.update(minus)
        last_size += minus

pbar.close()

100%|████████████████████████████████████████████████████████████████████████▉| 27065/27066 [2:22:28<00:00,  3.17it/s]


consumer 26 done
consumer 18 done
consumer 31 done
consumer 7 done
consumer 0 done
consumer 30 done
consumer 44 done
consumer 15 doneconsumer 34 done

consumer 17 done
consumer 39 done
consumer 41 done
consumer 6 done
consumer 8 done
consumer 11 doneconsumer 40 done

consumer 33 done
consumer 46 done
consumer 37 done
consumer 43 done
consumer 36 done
consumer 24 done
consumer 16 done
consumer 22 done
consumer 14 done
consumer 23 done
consumer 20 done
consumer 28 done
consumer 9 done
consumer 1 done
consumer 42 done
consumer 32 done
consumer 25 done
consumer 27 done
consumer 10 done
consumer 29 done
consumer 35 done
consumer 21 done
consumer 45 done
consumer 47 done
consumer 19 done
consumer 49 done
consumer 13 done
consumer 12 done
consumer 2 done
consumer 4 done
consumer 48 done
consumer 38 done
consumer 5 done
consumer 3 done


In [38]:
with open('mysoalan.com-llama3-8b.jsonl', 'w') as fopen_l:
    index = 0
    for i in range(len(filtered)):
        r = []
        for t in filtered[i]['texts']:
            filename = f'generate-qa/{index}.json'
            with open(filename) as fopen:
                d = json.load(fopen)
            r.append(d)
            index += 1

        filtered[i]['generated'] = r
        fopen_l.write(f'{json.dumps(filtered[i])}\n')

In [39]:
from huggingface_hub import HfApi
api = HfApi()

In [40]:
api.upload_file(
    path_or_fileobj='mysoalan.com-llama3-8b.jsonl',
    path_in_repo='mysoalan.com-llama3-8b.jsonl',
    repo_id='malaysia-ai/mysoalan.com',
    repo_type='dataset',
)

mysoalan.com-llama3-8b.jsonl:   0%|          | 0.00/46.8M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/malaysia-ai/mysoalan.com/commit/870bdbc88c0394c9f72d80862a7cb0cbf5e34ae4', commit_message='Upload mysoalan.com-llama3-8b.jsonl with huggingface_hub', commit_description='', oid='870bdbc88c0394c9f72d80862a7cb0cbf5e34ae4', pr_url=None, pr_revision=None, pr_num=None)